In [1]:
import torch
from torch import Tensor
from typing import Callable, List, Optional, Type, Union
import torch.nn as nn
from functools import partial

from torchvision.models.resnet import BasicBlock, Bottleneck, conv1x1


class BasicBlock(BasicBlock):
    def __init__(self, *args, **kwargs):
        self.skips = kwargs.pop("skips")
        super().__init__(*args, **kwargs)

    def forward(self, x: Tensor) -> Tensor:
        if self.skips:
            identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None and self.skips:
            identity = self.downsample(x)

        if self.skips:
            out += identity
        out = self.relu(out)

        return out


class Bottleneck(Bottleneck):
    def __init__(self, *args, **kwargs):
        self.skips = kwargs.pop("skips", True)
        super().__init__(*args, **kwargs)

    def forward(self, x: Tensor) -> Tensor:
        if self.skips:
            identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None and self.skips:
            identity = self.downsample(x)

        if self.skips:
            out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        width_scale: float = 1.0,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        skips: bool = True,
    ) -> None:
        super().__init__()
        # _log_api_usage_once(self)

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.scale_width = width_scale
        self.skips = skips

        self.inplanes = int(64 * width_scale)
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(
            3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(
            block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0]
        )
        self.layer3 = self._make_layer(
            block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1]
        )
        self.layer4 = self._make_layer(
            block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2]
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(int(512 * width_scale * block.expansion), num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:

        planes = int(planes * self.scale_width)

        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes,
                planes,
                stride,
                downsample,
                self.groups,
                self.base_width,
                previous_dilation,
                norm_layer,
                skips=self.skips,
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                    skips=self.skips,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def build_resnet(model_config, num_classes, dataset):

    backbone_type = model_config['backbone_type']
    only_features = model_config['only_features']
    batchnorm_layers = model_config['batchnorm_layers']
    width_scale = model_config['width_scale']
    skips = model_config['skips']
    modify_resnet = model_config['modify_resnet']

    # model = torchvision.models.__dict__[backbone_type](num_classes=num_classes)
    resnet = partial(
        ResNet, num_classes=num_classes, width_scale=width_scale, skips=skips
    )
    if not batchnorm_layers:
        resnet = partial(resnet, norm_layer=nn.Identity)
    match backbone_type:
        case "resnet18":
            model = resnet(BasicBlock, [2, 2, 2, 2])
        case "resnet34":
            model = resnet(BasicBlock, [3, 4, 6, 3])
        case "resnet50":
            model = resnet(Bottleneck, [3, 4, 6, 3])
        case "resnet101":
            model = resnet(Bottleneck, [3, 4, 23, 3])
        case "resnet152":
            model = resnet(Bottleneck, [3, 8, 36, 3])
        case _:
            raise ValueError(f"Unknown backbone type: {backbone_type}")

    if modify_resnet and (dataset == "cifar100" or dataset == "cifar10"):
        model.maxpool = torch.nn.Identity()
        model.conv1 = torch.nn.Conv2d(
            3, int(64 * width_scale), kernel_size=3, stride=1, padding=2, bias=False
        )
    if only_features:
        model.fc = torch.nn.Identity()

    if not batchnorm_layers:
        # turn off batch norm tracking stats and learning parameters
        for n, m in model.named_modules():
            if isinstance(m, torch.nn.BatchNorm2d):
                m.track_running_stats = False
                m.affine = False
                m.running_mean = None
                m.running_var = None

    renset_penultimate_layer_size = {
        "resnet18": int(512 * width_scale),
        "resnet34": int(512 * width_scale),
        "resnet50": int(2048 * width_scale),
        "resnet101": int(2048 * width_scale),
        "resnet152": int(2048 * width_scale),
    }
    model.penultimate_layer_size = renset_penultimate_layer_size[backbone_type]

    return model


def _forward_impl(self, x: Tensor) -> Tensor:
    # See note [TorchScript super()]
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x)
    x = torch.flatten(x, 1)

    return x


In [2]:
model_config = {'backbone_type': 'resnet18',
                    'only_features': False,
                    'batchnorm_layers': True,
                    'width_scale': 1.0,
                    'skips': True,
                    'modify_resnet': True}
model = build_resnet(model_config=model_config, num_classes=10, dataset="cifar10")

In [4]:
model(torch.randn(1, 3, 32, 32))

tensor([[ 0.4034,  0.6839, -0.5647,  0.4180, -0.0185,  0.6428, -0.2552, -0.8983,
          0.6912,  0.2330]], grad_fn=<AddmmBackward0>)

In [2]:
from src.modules.architectures.mm_resnets import build_mm_resnet

In [3]:
model_config =  {'backbone_type': 'resnet18',
                    'only_features': False,
                    'batchnorm_layers': True,
                    'width_scale': 1.0,
                    'skips': True,
                    'modify_resnet': True,
                    'wheter_concate': False,
                    'overlap': 0.0,}
model_params = {'model_config': model_config, 'num_classes': 10, 'dataset_name': 'cifar10'}

In [4]:
model = build_mm_resnet(**model_params)

In [6]:
model.net3

Sequential(
  (0): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [7]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 45.304MB


In [13]:
def model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    print('model size: {:.3f}MB'.format(size_all_mb))
    print(param_size, buffer_size)

In [14]:
model_size(model.net1)

model size: 2.613MB
2732288 7760


In [15]:
model_size(model.net2)

model size: 2.613MB
2732288 7760


In [16]:
model_size(model.net3)

model size: 40.059MB
41973760 30800


In [17]:
model_size(model)

model size: 45.304MB
47458856 46320


In [18]:
model.net3

Sequential(
  (0): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1